# Animal Sound Archive Audio Data Scraping

Source: https://www.museumfuernaturkunde.berlin/en/science/animal-sound-archive

In [71]:
dest_url = "https://www.tierstimmenarchiv.de/RefSys/Statistics.php"

In [72]:
# import packages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import json, time, os
import pandas as pd

In [73]:
# settings
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("window-size=1920x1080")
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": "/Users/rachelmoon/courses/fourthbrain-mlops/capstone_project/animal_sound_archive" 
    })

In [74]:
# function to visit page
def visit_page():
    browser = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
    browser.get(dest_url)
    time.sleep(3) # need time to load all web elements
    return browser


In [82]:
# initiate selenium driver
browser = visit_page()



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Driver [/Users/rachelmoon/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
<ipython-input-74-0e313e27f277>:3: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


## Task 1. Save the table as a csv file

In [57]:
# access target table
table = browser.find_element_by_css_selector("table[border='1']")

rows = table.find_elements_by_tag_name("tr")

# store feature names
col_names_obj = rows[0].find_elements_by_tag_name("th")
col_names = [col_name.text for col_name in col_names_obj]

col_names.append("Species Link") # we will also collect species hyperlink information

In [59]:
# collect information for each row
df_rows = []
for row in rows[1:-1]:
    df_row = {}
    
    cols = row.find_elements_by_tag_name("td")
    for i, col in enumerate(cols):
        # collect URL information
        if (i == 10): # Download 
            url = col.find_element_by_tag_name('a').get_attribute('href')
            df_row[col_names[i]] = url
            continue
        elif (i == 0): # Species - need both name and url
            url = col.find_element_by_tag_name('a').get_attribute('href')
            df_row[col_names[-1]] = url
        df_row[col_names[i]] = col.text
    df_rows.append(df_row)

In [66]:
# store results as dataframe and save as csv file
df = pd.DataFrame(df_rows)
df.to_csv('../src/data/animal-sound-archive.csv', index=False)

## Task 2. For each animal species, download audio data

In [83]:
# for each row, click download
queue = 0
for i, download_url in enumerate(df['Download Link']):
    if i < queue:
        continue
    browser.get(download_url)
    print(f"Downloading #{i}")
    time.sleep(20)

In [ ]:
# collect metadata, by also storing species link?
